In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


In [34]:
df = pd.read_csv("./table10.csv", index_col='Year')
df = df.groupby(by=df.index).sum().melt(ignore_index=False)
df = df.rename(columns={'variable':'Type', 'value': 'Number'})
display(df)
df_nototal = df[df.Type != 'Total']


natural_df = pd.read_excel("./naturalDisasters.xlsx")[['Year', 'Total Affected']]
natural_df = natural_df[natural_df.Year <= 2009]
natural_df = natural_df.groupby(by=natural_df.Year).sum().reset_index()
display(natural_df)

,Type,Number
Year,,
2000,Total,234456.0
2001,Total,277907.0
2002,Total,283838.0
2003,Total,168419.0
2004,Total,236697.0
...,...,...
2015,Other,16107.0
2016,Other,17299.0
2017,Other,19384.0


,Year,Total Affected
0,2000,158574.0
1,2001,2636465.0
2,2002,866486.0
3,2003,300118.0
4,2004,173804.0
5,2005,3681010.0
6,2006,297423.0
7,2007,2162884.0
8,2008,1354347.0
9,2009,3171027.0


In [48]:
df_bush = df_nototal[df_nototal.index < 2010]
fig = make_subplots(specs=[[{"secondary_y": True}]])

# fig = px.area(df_bush, x=df_bush.index, y='Number', color='Type',
#               color_discrete_sequence=px.colors.sequential.Blues,
#               title='LPR admittance under Bush following 9/11 Policies'
#              )
chart_max = (df['Number'].max())

# +(1/12*{month})
policies = {'USA Patriot Act': 2001+(1/12*10), 'Homeland Security Act of 2002': 2002+(1/12*3), 'Enhanced Border Security and Visa Entry Reform Act': 2002+(1/12*4), }

for policy, year in policies.items():
    fig.add_bar(x=[year], y=[chart_max],
                name=policy, width=0.1,
                hovertemplate=policy,
                marker_color='#b60028'
               )
    
for i, type in enumerate(df_bush['Type'].unique()):
    fig.add_trace(go.Scatter(
        x=df_bush[df_bush.Type == type].index, y=df_bush[df_bush.Type == type].Number,
        mode='lines',
        name=type,
        line=dict(width=0.5, color=px.colors.sequential.Blues[i]),
        stackgroup='one' # define stack group
    ))

# display(natural_df['Year'])
fig.add_trace(go.Scatter(x=natural_df.Year, y=natural_df['Total Affected'], name='People affected by natural disasters',),
             
              secondary_y=True)

fig.update_layout({
    'title': 'LPR admittance under Bush following 9/11 Policies',
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})
fig.show()

In [49]:
fig.write_html("./9-11.txt", include_plotlyjs=False, full_html=False)

In [5]:
lpr_fig = px.area(df_nototal, x=df_nototal.index, y='Number', color='Type',
              color_discrete_sequence=px.colors.sequential.Blues,
              title='LPR admittance (2000-2019)'
             )

lpr_fig.add_vrect(x0="2001", x1="2009", 
              annotation_text="Bush", annotation_position="top left",
              fillcolor="#999999", opacity=0.25, line_width=0)

lpr_fig.add_vrect(x0="2009", x1="2017", 
              annotation_text="Obama", annotation_position="top left",
              opacity=0.25, line_width=0)

lpr_fig.add_vrect(x0="2017", x1="2019", 
              annotation_text="Trump", annotation_position="top left",
              fillcolor="#999999", opacity=0.25, line_width=0)

lpr_fig.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})

lpr_fig.show()

In [6]:
lpr_fig.write_html("./lpr.txt", include_plotlyjs=False, full_html=False)